# Introdução

Para o presente projeto, é necessário fazer o *download* os dados espaciais contendo a divisão municipal. Optou-se por baixar os dados do [**Instituto Brasileiro de Geografia e Estatistica (IBGE)**](https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais), em escala 1:250.000, suficiente para atender o objetivo do projeto, que trata-se de demostrar quais as divisões administrativas, por município, de diversos órgãos e/ou instituições.

Portanto, faz-se o *download* do arquivo zipado do **IBGE** e se descompacta o arquivo _shapefile_. Aproveitou-se para renomear o arquivo um nome mais inteligível.

[Repo SP](https://github.com/michelmetran/sp)

In [ ]:
import os
import requests
import pandas as pd
import geopandas as gpd
from zipfile import ZipFile

In [ ]:
#!pip install openpyxl

In [ ]:
# Create Directory
os.makedirs('data/tabs', exist_ok=True)
os.makedirs('data/shps', exist_ok=True)

# Functions

In [ ]:
def get_download(url, filename, path):
    # Define o nome do arquivo, com diretório, que será salvo
    zipfile = os.path.join(path, filename)

    # Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
    r = requests.get(url, stream=True)
    with open(zipfile, 'wb') as out:
        for chunk in r.iter_content(chunk_size=128):
            out.write(chunk)
    print('File "{}" download in "{}" directory.'.format(filename, path))

In [ ]:
def unzip(filename, path):
    # Get dir of zipfile
    zipfile = os.path.join(path, filename)
    path = os.path.dirname(zipfile)

    # Create a ZipFile Object
    with ZipFile(zipfile, 'r') as zip_obj:
        # Get a list of all archived file names from the zip
        list_files = []
        list_files = zip_obj.namelist()

        # Iterate over the file names
        for file in list_files:
            name, ext = os.path.splitext(file)
            zip_obj.extract(file, os.path.dirname(zipfile))
            os.rename(os.path.join(path, file), os.path.join(path, os.path.splitext(filename)[0]+ext))
            print('File "{}" extracted as "{}" in "{}" directory.'.format(file, os.path.splitext(filename)[0]+ext, path))

    # Remove file
    os.remove(zipfile)

# Limite Municipal 1:250.000

In [ ]:
# Input
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/UFs/SP/SP_Municipios_2020.zip'
filename = 'sp_ibge_250k.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
# Converte em *geojson*
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop(['NM_MUN', 'SIGLA_UF', 'AREA_KM2'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(columns={'CD_MUN': 'id_municipio'}, inplace=True)

# Results
display(gdf.head())

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Salva Arquivo
gdf.to_file(
    os.path.join('data', 'shps', 'sp_250k_wgs84.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

In [ ]:
# Cria Cópia
df = gdf[['id_municipio']]

# Pega apenas os três primeiros caracteres de um campo de número de maneira mais eficiente
df['id_microregiao'] = [x[0:5] for x in df['id_municipio']]
df.head()

In [ ]:
# Cria Cópia
df = gdf[['id_municipio']]

# Pega apenas os três primeiros caracteres de um campo de número de maneira mais eficiente
df['id_mesoregiao'] = [x[0:4] for x in df['id_municipio']]
df.head()

In [ ]:



gdf['id_estado'] = [x[0:2] for x in gdf['id_municipio']]



# Limite Municipal 1:50.000

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/LimiteMunicipal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=LimiteMunicipal'
filename = 'sp_igc_50k.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
file = 'sp_igc_50k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
#gdf.drop(['NM_MUN', 'SIGLA_UF', 'AREA_KM2'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
        'Nome'      : 'rotulo_municipio',
        'Rotulo'    : 'nome_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    }, inplace=True)

# Results
display(gdf.head())

In [ ]:
# Cria Cópia
df = gdf

# Seleciona
df[['id_municipio', 'id_cetesb']]

In [ ]:
# Cria Cópia
df = gdf

# Seleciona
df[['id_municipio', 'id_ugrhi', 'nome_ugrhi']]

In [ ]:
set(df['nome_ugrhi'])

# Limite Macrometropole

TODO: Dar um Insersect para pegar apenas a tabela de atributos dos municípios.

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/LimiteMacrometropole2015/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=LimiteMacrometropole2015'
filename = 'sp_macrometropole.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
file = 'sp_macrometropole'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Results
display(gdf.head())

# Regiões Metropolitanas e Aglomerações Urnanas

In [ ]:
# URL para baixar o arquivo zip
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/municipios_por_regioes_metropolitanas/Situacao_2020a2029/Composicao_RMs_RIDEs_AglomUrbanas_2020_06_30.xlsx'

# Read File
df = pd.read_excel(
    url,
    engine='openpyxl',
)
print('Existem as seguintes colunas disponíveis:\n{}\n'.format(list(df.columns)))

df = df.loc[df['COD_UF'] == 35]
df = df[['COD_MUN', 'NOME', 'LEG', 'DATA']]
df.reset_index(drop=True, inplace=True)

df.rename(
    columns={
        'COD_MUN': 'id_municipio',
        'NOME'   : 'nome_rm',
        'LEG'    : 'legislacao',
        'DATA'   : 'data',
    },
    inplace=True,
)

# 
df['data_format'] = df['data'].dt.strftime('%d.%m.%Y')
df['legislacao_data'] = df[['legislacao', 'data_format']].apply(lambda x: ' de '.join(x), axis=1)
df.drop('data_format', axis=1, inplace=True)

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_rms.csv'),
    index=False
)
df.head()

___

# Encerramento

## *Requirements*

In [ ]:
# Input
inp = 'get_spatial_data.ipynb'

# Output
out = os.path.join(inp.split('.')[0])
!jupyter-nbconvert $inp --to python --output $out

print('-'*80)
!pipreqs '.' --force --debug

print('-'*80)
os.remove('{}.py'.format(out))
print('Arquivo {}.py deletado com sucesso!'.format(out))

## *GitHub*

In [ ]:
!nbstripout --install --attributes .gitattributes
!git status
!git add .
!git commit -m 'Arrumações Gerais'
!git push origin main